In [1]:
# imports
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"


In [2]:
# read csv
df = pd.read_csv(filepath_or_buffer='tornados.csv', low_memory=False, parse_dates=['datetime_utc'], index_col=['om'])

In [3]:
df.head()

,yr,mo,dy,date,time,tz,datetime_utc,st,stf,mag,...,elon,len,wid,ns,sn,f1,f2,f3,f4,fc
om,,,,,,,,,,,,,,,,,,,,,
192,1950,10,1,1950-10-01,21:00:00,America/Chicago,1950-10-02 03:00:00+00:00,OK,40,1.0,...,-102.3,15.8,10,1,1,25,0,0,0,False
193,1950,10,9,1950-10-09,02:15:00,America/Chicago,1950-10-09 08:15:00+00:00,NC,37,3.0,...,0.0,2.0,880,1,1,47,0,0,0,False
195,1950,11,20,1950-11-20,02:20:00,America/Chicago,1950-11-20 08:20:00+00:00,KY,21,2.0,...,0.0,0.1,10,1,1,177,0,0,0,False
196,1950,11,20,1950-11-20,04:00:00,America/Chicago,1950-11-20 10:00:00+00:00,KY,21,1.0,...,0.0,0.1,10,1,1,209,0,0,0,False
197,1950,11,20,1950-11-20,07:30:00,America/Chicago,1950-11-20 13:30:00+00:00,MS,28,1.0,...,0.0,2.0,37,1,1,101,0,0,0,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68693 entries, 192 to 621901
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   yr            68693 non-null  int64              
 1   mo            68693 non-null  int64              
 2   dy            68693 non-null  int64              
 3   date          68693 non-null  object             
 4   time          68693 non-null  object             
 5   tz            68693 non-null  object             
 6   datetime_utc  68693 non-null  datetime64[ns, UTC]
 7   st            68693 non-null  object             
 8   stf           68693 non-null  int64              
 9   mag           67937 non-null  float64            
 10  inj           68693 non-null  int64              
 11  fat           68693 non-null  int64              
 12  loss          41523 non-null  float64            
 13  slat          68693 non-null  float64            
 14  slon    

In [5]:
from plotly.express import bar
bar(data_frame=df['st'].value_counts().to_frame().reset_index(), x='st', y='count')

In [6]:
# show map of tornados in ohio
fig = px.scatter_mapbox(df[df['st'] == 'OH'], lat='slat', lon='slon', hover_name='datetime_utc', hover_data=['mag'], zoom=5, height=300)
fig.update_layout(mapbox_style='open-street-map')
fig.show()